## Docs Source: 
### 1 : https://python.langchain.com/docs/introduction/

In [8]:
#!pip install arxiv

In [9]:
#!pip install langchain_community

In [10]:
import os
import dotenv 
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [11]:
api_wrapper=WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200) # WikipediaAPIWrapper: configure API of wikipedia, top_k_results: How many results i want to see, doc_content_chars_max: how many numbers of charactors i want to get in the response.
wiki=WikipediaQueryRun(api_wrapper=api_wrapper) # WikipediaQueryRun: this is the main Tool.

In [12]:
# Loadiung the Enviroment Variables
from dotenv import load_dotenv
load_dotenv()

True

In [13]:
wiki.name

'wikipedia'

In [14]:
from langchain_community. document_loaders import WebBaseLoader
from langchain_community. vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)
vectordb=FAISS.from_documents(documents, OpenAIEmbeddings(api_key=os.environ["OPENAI_API_KEYS"])) # FAISS (Facebook AI Similarity Search)
retriever=vectordb.as_retriever()
retriever

USER_AGENT environment variable not set, consider setting it to identify your requests.


VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002CDFF50DFF0>, search_kwargs={})

In [15]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(retriever, "langsmith_search","search for information about LangSmith. For any questions about LangSmith, you must use this tool.") # create_retriever_tool: If I want to use the FAISS as a tool so i have to use it as a tool, Docs: https://api.python.langchain.com/en/latest/tools/langchain.tools.retriever.create_retriever_tool.html . 

In [16]:
retriever_tool.name

'langsmith_search'

In [17]:
## Arxiv Tool
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper=ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv=ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

In [18]:
# Combining tools and the langchain qury one by one as the sequence i have add it in.

tools=[wiki, arxiv, retriever_tool]

In [19]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'w:\\Bard Dialogflow ES connection with simple HTML  and JS website using cloud function\\Langchain fast server\\notebook\\venv\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200)),
 Tool(name='langsmith_search', description='search for information about LangSmith. For any questions about LangSmith, you must use this tool.', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000002CDE2B22710>, retriev

In [20]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'w:\\Bard Dialogflow ES connection with simple HTML  and JS website using cloud function\\Langchain fast server\\notebook\\venv\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200)),
 Tool(name='langsmith_search', description='search for information about LangSmith. For any questions about LangSmith, you must use this tool.', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000002CDE2B22710>, retriev

In [22]:

from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0,api_key=os.environ["OPENAI_API_KEYS"])

In [23]:
from langchain import hub
# Get the prompt from langchain hub for the openAI Agent, so we don't have to thing and write it down to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages # prompt.messages: to see the prompt

w:\Bard Dialogflow ES connection with simple HTML  and JS website using cloud function\Langchain fast server\notebook\venv\lib\site-packages\langsmith\client.py:354: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [25]:
### Agents
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm, tools, prompt)

In [26]:
## Agent Execiter
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent,tools=tools,verbose=True) # Verbose Ture so i can see all deatils about from where it is getting the response etc.
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [28]:
agent_executor.invoke({"input":"Tell me about machine learning"})




> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Machine Learning'}`


Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and genMachine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generate insights or predictions. It involves creating models that can automatically learn and improve from experience without being explicitly programmed. Machine learning algorithms are used in various applications such as image recognition, natural language processing, and recommendation systems.

> Finished chain.


{'input': 'Tell me about machine learning',
 'output': 'Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generate insights or predictions. It involves creating models that can automatically learn and improve from experience without being explicitly programmed. Machine learning algorithms are used in various applications such as image recognition, natural language processing, and recommendation systems.'}